In [1]:
import os
import pandas as pd

# Functions to compute the numerical results

In [2]:
def load_metrics(result_folder, split):
    if "LFLR" in result_folder:
        metrics_file = os.path.join(result_folder, "LR", f"split_{split}", f"metrics_split{split}.csv")
    elif "LFCO" in result_folder:
        metrics_file = os.path.join(result_folder, "LR", f"split_{split}", f"metrics_split{split}.csv")
    else:
        metrics_file = os.path.join(result_folder, f"split_{split}", f"metrics_split{split}.csv")
    
    if not os.path.exists(metrics_file):
        raise FileNotFoundError(f"File not found: {metrics_file}")
    
    return pd.read_csv(metrics_file)

# Function to calculate the mean and standard deviation of metrics
def calculate_statistics(result_folder, splits=[1, 2, 3]):
    all_metrics = []

    # Load metrics from all splits
    for split in splits:
        split_metrics = load_metrics(result_folder, split)
        all_metrics.append(split_metrics)

    # Concatenate all split metrics into a DataFrame
    metrics_df = pd.concat(all_metrics)

    # Calculate mean and standard deviation for each metric
    statistics = {
        "mean": metrics_df.mean() * 100,
        "std_dev": metrics_df.std() * 100
    }

    return statistics

def get_valid_subfolders(directory):
    return sorted([f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f)) and not f.startswith('.')])

# Results

In [3]:
# models = ["./MLP", "./GRU", "./JHF", "./FHSI", "./LFLR", "./LFCO"]
models = ["./MLP", "./JHF", "./GRU", './LFCO', './FHSI', './LFLR']

results_list = []

for model in models:
    result_subfolders = get_valid_subfolders(model)
    
    for result_folder in result_subfolders:
        result_path = os.path.join(model, result_folder)

        try:
            statistics = calculate_statistics(result_path)
            mean = statistics["mean"]
            std_dev = statistics["std_dev"]
            row = {
                "Model": f"{os.path.basename(model)}_{result_folder}",
                "Accuracy": f"{mean['accuracy']:.2f} ± {std_dev['accuracy']:.2f}",
                "Specificity": f"{mean['specificity']:.2f} ± {std_dev['specificity']:.2f}",
                "Sensitivity": f"{mean['recall']:.2f} ± {std_dev['recall']:.2f}",
                "ROC AUC": f"{mean['roc_auc']:.2f} ± {std_dev['roc_auc']:.2f}"
            }
            
            results_list.append(row)
        
        except FileNotFoundError as e:
            print(e)

results_df = pd.DataFrame(results_list)
results_df = results_df.sort_values(by="Model").reset_index(drop=True)

In [4]:
results_df

,Model,Accuracy,Specificity,Sensitivity,ROC AUC
0,FHSI_Results_FHSI-3MTS-3Features,69.41 ± 0.60,68.61 ± 0.61,72.54 ± 0.99,74.67 ± 0.96
1,FHSI_Results_FHSI-4MTS-3Features,71.47 ± 0.46,70.59 ± 0.20,74.96 ± 1.71,76.49 ± 2.25
2,FHSI_Results_FHSI-5MTS-3Features,69.57 ± 3.39,69.01 ± 4.95,71.89 ± 3.31,76.90 ± 3.10
3,GRU_Results_GRU-3MTS,71.68 ± 2.36,73.00 ± 3.82,66.28 ± 4.64,73.42 ± 1.40
4,GRU_Results_GRU-4MTS,67.46 ± 4.31,66.96 ± 5.68,69.17 ± 3.21,71.86 ± 1.85
5,GRU_Results_GRU-5MTS,69.25 ± 1.35,69.86 ± 0.59,67.03 ± 4.62,72.47 ± 1.56
6,JHF_Results_JHF-3MTS-3Features,69.04 ± 2.72,68.85 ± 5.11,70.29 ± 7.68,75.14 ± 1.47
7,JHF_Results_JHF-4MTS-3Features,69.67 ± 3.21,70.25 ± 5.27,67.85 ± 5.45,74.30 ± 0.65
8,JHF_Results_JHF-5MTS-3Features,69.67 ± 0.71,70.12 ± 1.02,67.91 ± 1.17,73.69 ± 0.42
9,LFCO_Results_LFCO-3MTS-3Features,57.86 ± 3.09,55.88 ± 3.39,65.66 ± 3.63,65.56 ± 2.36


In [5]:
results_df['Accuracy'].max()

'71.68 ± 2.36'

In [6]:
results_df['Specificity'].max()

'73.00 ± 3.82'

In [7]:
results_df['Sensitivity'].max()

'74.96 ± 1.71'

In [8]:
results_df['ROC AUC'].max()

'76.90 ± 3.10'

In [9]:
for col in ['Accuracy', 'Specificity', 'Sensitivity', 'ROC AUC']:
    results_df[[f'{col}_Mean', f'{col}_StdDev']] = results_df[col].str.split(' ± ', expand=True).astype(float)

mean_values = results_df[[f'{col}_Mean' for col in ['Accuracy', 'Specificity', 'Sensitivity', 'ROC AUC']]].mean()
mean_stddevs = results_df[[f'{col}_StdDev' for col in ['Accuracy', 'Specificity', 'Sensitivity', 'ROC AUC']]].mean()

print("Mean of the values:")
print(mean_values)
print("\nMean of the standard deviations:")
print(mean_stddevs)

Mean of the values:
Accuracy_Mean       65.310000
Specificity_Mean    64.694444
Sensitivity_Mean    67.738333
ROC AUC_Mean        71.119444
dtype: float64

Mean of the standard deviations:
Accuracy_StdDev       2.341111
Specificity_StdDev    3.519444
Sensitivity_StdDev    4.558889
ROC AUC_StdDev        1.521111
dtype: float64
